In [2]:
!pip install nltk

DEPRECATION: Loading egg at /opt/bitnami/python/lib/python3.11/site-packages/pip-23.3.2-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 15.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 785.0/785.0 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 12.7 MB/s eta 0:00:00


In [3]:
import nltk

# Initialize NLTK's Vader and download stopwords
nltk.download('vader_lexicon')
nltk.download('stopwords')

[nltk_data] Downloading package vader_lexicon to /nltk_data...
[nltk_data] Downloading package stopwords to /nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# references = df.select(
#                 explode(split(col("filtered_title"), " ")).alias("word")
#             ).filter(
#                 (col("word").like("/u/%")) |
#                 (col("word").like("/r/%")) |
#                 (col("word").like("http%"))
#             )
#             print("-----------------------------")
#             print(references.collect())
#             print("-----------------------------")
#             # User references
#             user_refs = references.filter(col("word").like("/u/%")).groupBy("word").count().withColumnRenamed("count", "user_references_count")
#             user_refs.createOrReplaceTempView("user_references")
#             user_refs.write.json("/data/output/user_references", mode="append")
#             print("User references table created and saved.")

#             # Post references
#             post_refs = references.filter(col("word").like("/r/%")).groupBy("word").count().withColumnRenamed("count", "post_references_count")
#             post_refs.createOrReplaceTempView("post_references")
#             post_refs.write.json("/data/output/post_references", mode="append")
#             print("Post references table created and saved.")
            
#             # URL references
#             url_refs = references.filter(col("word").like("http%")).groupBy("word").count().withColumnRenamed("count", "url_references_count")
#             url_refs.createOrReplaceTempView("url_references")
#             url_refs.write.json("/data/output/url_references", mode="append")
#             print("URL references table created and saved.")

In [ ]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import explode, split, col, udf, min, max, lit,  lower
from pyspark.sql.types import StringType, DoubleType
from pyspark.ml.feature import HashingTF, IDF
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
import json
import os

sid = SentimentIntensityAnalyzer()
stop_words = set(stopwords.words('english'))

# Define the host and port
host = "127.0.0.1"
port = 9998

# Initialize SparkContext and SparkSession
sc = SparkContext("local[2]", "RedditConsumer")
sc.setLogLevel("ERROR")
spark = SparkSession(sc)

# Initialize StreamingContext with a batch interval of 10 seconds
ssc = StreamingContext(sc, 60)

# Ensure the necessary directories exist
os.makedirs("/data/raw/reddit_ask", exist_ok=True)
os.makedirs("/data/output/user_references", exist_ok=True)
os.makedirs("/data/output/post_references", exist_ok=True)
os.makedirs("/data/output/url_references", exist_ok=True)
os.makedirs("/data/output/top_words", exist_ok=True)
os.makedirs("/data/output/metrics", exist_ok=True)
os.makedirs("/data/output/sentiment_analysis", exist_ok=True)
def normalize_text(text):
    text = text.lower()  
    text = text.replace("’s", "")  
    text = text.replace("whats", "what")
    return text
def get_sentiment(text):
    scores = sid.polarity_scores(text)
    return scores['compound']

def classify_sentiment(polarity):
    if polarity > 0.05:
        return 'positive'
    elif polarity < -0.05:
        return 'negative'
    else:
        return 'neutral'

def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

get_sentiment_udf = udf(get_sentiment, DoubleType())
classify_sentiment_udf = udf(classify_sentiment, StringType())
remove_stopwords_udf = udf(remove_stopwords, StringType())
def new_columns(df):
    # Remove stopwords from titles
    df = df.withColumn("filtered_title", remove_stopwords_udf(col("title")))
    df = df.withColumn("polarity", get_sentiment_udf(col("filtered_title")))
    df = df.withColumn("sentiment", classify_sentiment_udf(col("polarity")))
    return df
def tf_idf(df):
    words_data = df.select(explode(split(col("filtered_title"), " ")).alias("words"))
    hashing_tf = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=20)
    idf = IDF(inputCol="rawFeatures", outputCol="features")
    tf = hashing_tf.transform(words_data)
    idf_model = idf.fit(tf)
    return idf_model.transform(tf)
def time_optional(df):
    # Get the time range of the data
    time_range = df.select(min("date").alias("start_time"), max("date").alias("end_time")).collect()
    if time_range:
        start_time = time_range[0]["start_time"]
        end_time = time_range[0]["end_time"]
        print(f"Time range: {start_time} to {end_time}")
def metric(user_refs):
    #optional
        # Store metrics in a temporary table
    metrics_df = user_refs.join(post_refs, "word", "outer") \
                          .join(url_refs, "word", "outer")
    metrics_df = metrics_df.withColumn("average_sentiment", lit(average_sentiment))
    metrics_df = metrics_df.withColumn("start_time", lit(start_time))
    metrics_df = metrics_df.withColumn("end_time", lit(end_time))
    metrics_df.createOrReplaceTempView("metrics")
    metrics_df.write.json("/data/output/metrics", mode="append")
    print("Metrics table created and saved.")
    metrics_df.show(5, truncate=False)

def placeholder():
    # Calculate TF-IDF
            
    tfidf_data = tf_idf(df)
    print(tfidf_data)
    # Find the top words and their TF-IDF scores
    top_words = tfidf_data.select("words", "features").rdd.flatMap(
        lambda row: [(row.words, float(v)) for v in row.features.toArray()]
    ).sortBy(lambda x: -x[1]).take(10)
    print(top_words.collect())
    # Convert top words and their TF-IDF scores to DataFrame and persist it
    top_words_df = spark.createDataFrame(top_words, ["word", "tfidf"])
    top_words_df.createOrReplaceTempView("top_words")
    #top_words_df.write.json("/data/output/top_words", mode="append")
    print(spark.sql("select * from top_words"))
    print("TF-IDF table created and saved.")
    
    # Sentiment Analysis using NLTK's Vader
    sentiment_df = df.select("title", "polarity", "sentiment")
    sentiment_df.createOrReplaceTempView("sentiment_analysis")
    sentiment_df.write.json("/data/output/sentiment_analysis", mode="append")
    print("Sentiment analysis table created and saved.")
    
    sentiment_rdd = df.select("polarity").rdd.map(lambda row: row.polarity)
    average_sentiment = sentiment_rdd.mean()
    print(f"Average sentiment: {average_sentiment}")

def process_rdd(rdd):
    stopwords = [
    "a", "an", "and", "are", "as", "at", "be", "but", "by", "for", "if", "in", "into", "is", "it", "no", "not", "of",
    "on", "or", "such", "that", "the", "their", "then", "there", "these", "they", "this", "to", "was", "will", "with"
    ]
    """
    Process each RDD:
    1. Convert RDD to DataFrame and persist it.
    2. Extract references to users, posts, and URLs.
    3. Calculate TF-IDF.
    4. Find the top words and their TF-IDF scores.
    5. Perform sentiment analysis.
    6. Store metrics in a temporary table.
    7. Get the time range of the data.
    """
    print("Processing RDD...")
    if not rdd.isEmpty():
        # Parse each record as JSON
        records = rdd.map(lambda record: json.loads(record)).collect()
        
        # Check if records are valid and not empty
        if records:
            # Convert RDD to DataFrame
            df = spark.createDataFrame(records)
            df = new_columns(df)
          
            # Save raw data
            df.createOrReplaceTempView("raw")
            df.write.json("/data/raw/reddit_ask", mode="append")
            
            # Print DataFrame schema and some data
            #df.printSchema()
            df = df.dropDuplicates()
            df.show( truncate=False)
            # Top 10 words
            print("Word count for window")
            normalize_udf = udf(lambda x: normalize_text(x))
            words_df = df.withColumn("normalized_title", normalize_udf(col("filtered_title"))) \
             .select(explode(split(col("normalized_title"), " ")).alias("word"))
            words_filtered_df = words_df.filter(~col("word").isin(stopwords))
            word_count_df = words_filtered_df.groupBy("word").count().orderBy("count",ascending=False)
            word_count_df.show(10)

            
        else:
            print("No valid records found in this RDD.")

# Define the socket stream
lines = ssc.socketTextStream(host, port)

# Apply the processing function to each RDD in the DStream
lines.foreachRDD(lambda rdd: process_rdd(rdd))

# Print received lines (for debugging)
#lines.pprint()

# Start the streaming context and wait for termination
ssc.start()
print("Streaming started...")
ssc.awaitTermination()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/09 23:29:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/opt/bitnami/spark/python/pyspark/streaming/context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(


Streaming started...


24/06/09 23:29:10 ERROR ReceiverTracker: Deregistered receiver for stream 0: Restarting receiver with delay 2000ms: Socket data stream had no more data
24/06/09 23:29:13 ERROR ReceiverTracker: Deregistered receiver for stream 0: Restarting receiver with delay 2000ms: Socket data stream had no more data
24/06/09 23:29:16 ERROR ReceiverTracker: Deregistered receiver for stream 0: Restarting receiver with delay 2000ms: Socket data stream had no more data
24/06/09 23:29:19 ERROR ReceiverTracker: Deregistered receiver for stream 0: Restarting receiver with delay 2000ms: Socket data stream had no more data
24/06/09 23:29:22 ERROR ReceiverTracker: Deregistered receiver for stream 0: Restarting receiver with delay 2000ms: Socket data stream had no more data
24/06/09 23:29:25 ERROR ReceiverTracker: Deregistered receiver for stream 0: Restarting receiver with delay 2000ms: Socket data stream had no more data
24/06/09 23:29:28 ERROR ReceiverTracker: Deregistered receiver for stream 0: Restarting 

Processing RDD...


24/06/09 23:30:01 ERROR ReceiverTracker: Deregistered receiver for stream 0: Restarting receiver with delay 2000ms: Socket data stream had no more data
24/06/09 23:30:04 ERROR ReceiverTracker: Deregistered receiver for stream 0: Restarting receiver with delay 2000ms: Socket data stream had no more data


...

24/06/09 23:38:02 ERROR ReceiverTracker: Deregistered receiver for stream 0: Restarting receiver with delay 2000ms: Socket data stream had no more data
24/06/09 23:38:05 ERROR ReceiverTracker: Deregistered receiver for stream 0: Restarting receiver with delay 2000ms: Socket data stream had no more data
24/06/09 23:38:08 ERROR ReceiverTracker: Deregistered receiver for stream 0: Restarting receiver with delay 2000ms: Socket data stream had no more data
24/06/09 23:38:11 ERROR ReceiverTracker: Deregistered receiver for stream 0: Restarting receiver with delay 2000ms: Socket data stream had no more data
24/06/09 23:38:13 ERROR ReceiverTracker: Deregistered receiver for stream 0: Restarting receiver with delay 2000ms: Socket data stream had no more data
24/06/09 23:38:16 ERROR ReceiverTracker: Deregistered receiver for stream 0: Restarting receiver with delay 2000ms: Socket data stream had no more data
24/06/09 23:38:19 ERROR ReceiverTracker: Deregistered receiver for stream 0: Restarting 

Processing RDD...


24/06/09 23:39:01 ERROR ReceiverTracker: Deregistered receiver for stream 0: Restarting receiver with delay 2000ms: Socket data stream had no more data


+------------------+-------------------+---------+----------------------------------------------------------------------+------------------------------------------+--------+---------+
|author            |date               |subreddit|title                                                                 |filtered_title                            |polarity|sentiment|
+------------------+-------------------+---------+----------------------------------------------------------------------+------------------------------------------+--------+---------+
|Small-Support-3677|2024-06-09 23:08:54|AskReddit|How do you feel about having palms sweaty, knees weak, and arms heavy?|feel palms sweaty, knees weak, arms heavy?|-0.4404 |negative |
|RateChemical8339  |2024-06-09 23:09:19|ask      |what is an app i can use to organize my clothes?                      |app use organize clothes?                 |0.0     |neutral  |
+------------------+-------------------+---------+------------------------------

24/06/09 23:39:04 ERROR ReceiverTracker: Deregistered receiver for stream 0: Restarting receiver with delay 2000ms: Socket data stream had no more data
24/06/09 23:39:07 ERROR ReceiverTracker: Deregistered receiver for stream 0: Restarting receiver with delay 2000ms: Socket data stream had no more data
24/06/09 23:39:09 ERROR ReceiverTracker: Deregistered receiver for stream 0: Restarting receiver with delay 2000ms: Socket data stream had no more data
24/06/09 23:39:12 ERROR ReceiverTracker: Deregistered receiver for stream 0: Restarting receiver with delay 2000ms: Socket data stream had no more data
24/06/09 23:39:15 ERROR ReceiverTracker: Deregistered receiver for stream 0: Restarting receiver with delay 2000ms: Socket data stream had no more data
24/06/09 23:39:18 ERROR ReceiverTracker: Deregistered receiver for stream 0: Restarting receiver with delay 2000ms: Socket data stream had no more data
24/06/09 23:39:21 ERROR ReceiverTracker: Deregistered receiver for stream 0: Restarting 